First, we're going to use Scryfall's wonderful API to download all of the cards and their features. Because there are a limited number of cards & features, it should be OK to save these all in .csv for now. 

In [2]:
# Import basic data wrangling and scraping tools 
import numpy as np
import pandas as pd
import json, requests, pickle
from bs4 import BeautifulSoup

## Exploring set data

Before jumping into the cards (which are a little more complicated), let's start by looking at the sets (card expansions).

In [6]:
# Get setlist into dataframe
link = 'https://api.scryfall.com/sets'
response = requests.get(link)
sets = response.json()['data']
sets_df = pd.DataFrame(sets)
sets_df.head(1)

,block,block_code,card_count,code,digital,foil_only,icon_svg_uri,mtgo_code,name,object,parent_set_code,released_at,scryfall_uri,search_uri,set_type,uri
0,NaN,NaN,40,puma,False,True,https://img.scryfall.com/sets/uma.svg?1541998800,NaN,Ultimate Box Topper,set,uma,NaN,https://scryfall.com/sets/puma,https://api.scryfall.com/cards/search?order=se...,masterpiece,https://api.scryfall.com/sets/puma



Looks like there is a lot of extra data here we don't need. Let's clean it up a bit:

In [7]:
# Ignore sets with no release date (likely promotional / outliers)
sets_df.dropna(subset=['released_at'], inplace=True)
# Set release date to index - chronological order makes sense, right?
sets_df.set_index('released_at', inplace=True)
# Remove repetitive information and online data
sets_df.drop(['digital','mtgo_code','parent_set_code','object'], axis=1, inplace=True)

sets_df.head(1)

,block,block_code,card_count,code,foil_only,icon_svg_uri,name,scryfall_uri,search_uri,set_type,uri
released_at,,,,,,,,,,,
2018-12-07,NaN,NaN,41,uma,False,https://img.scryfall.com/sets/uma.svg?1541998800,Ultimate Masters,https://scryfall.com/sets/uma,https://api.scryfall.com/cards/search?order=se...,masters,https://api.scryfall.com/sets/uma


Looks better, but still some extra information here, like the uris. We might need them later though, so for the next cleaning step I'll just make a new dataframe.

In [16]:
# Make even cleaner for ease of exploration
clean_df = sets_df[['name','code','block_code','card_count','set_type']]
clean_df.head(10)

,name,code,block_code,card_count,set_type
released_at,,,,,
2018-12-07,Ultimate Masters,uma,NaN,41,masters
2018-11-16,Game Night,gnt,NaN,68,box
2018-11-02,GRN Guild Kit,gk1,NaN,127,box
2018-10-05,Guilds of Ravnica,grn,grn,273,expansion
2018-08-09,Commander 2018,c18,NaN,307,commander
2018-08-01,2017 Heroes of the Realm,htr17,NaN,1,memorabilia
2018-07-14,Arena New Player Experience,ana,NaN,41,starter
2018-07-13,Core Set 2019,m19,NaN,314,core
2018-06-22,Global Series Jiang Yanggu & Mu Yanling,gs1,NaN,41,duel_deck


That's better, but we notice that a lot of these sound weird or have low card counts. Let's investigate the mysterious "set_type":

In [10]:
clean_df['set_type'].unique()

array(['masters', 'box', 'expansion', 'commander', 'memorabilia',
       'starter', 'core', 'duel_deck', 'spellbook', 'draft_innovation',
       'funny', 'from_the_vault', 'archenemy', 'planechase',
       'treasure_chest', 'premium_deck'], dtype=object)

After doing some research on the sets, I found that a lot of them are promotional and not legal for any tournament format, or are only available online. So let's get rid of those:

In [17]:
# after extensive manual checking of set legality / onlineness
sets_to_drop = [
    'me1',
    'me2',
    'me3',
    'me4',
    'vma',
    'tpr',
    'e02',
    'gnt',
    'td0',
    'mgb',
    'ana',
    'w17',
    'w16',
    'itp'
]
types_to_drop = [
    'memorabilia',
    'funny',
    'treasure_chest'
]

final_sets = clean_df[clean_df['code'].apply(lambda x: x not in sets_to_drop)]
final_sets = final_sets[final_sets['set_type'].apply(lambda x: x not in types_to_drop)]
final_sets.head(10)

,name,code,block_code,card_count,set_type
released_at,,,,,
2018-12-07,Ultimate Masters,uma,NaN,41,masters
2018-11-02,GRN Guild Kit,gk1,NaN,127,box
2018-10-05,Guilds of Ravnica,grn,grn,273,expansion
2018-08-09,Commander 2018,c18,NaN,307,commander
2018-07-13,Core Set 2019,m19,NaN,314,core
2018-06-22,Global Series Jiang Yanggu & Mu Yanling,gs1,NaN,41,duel_deck
2018-06-15,Signature Spellbook: Jace,ss1,NaN,8,spellbook
2018-06-08,Battlebond,bbd,NaN,256,draft_innovation
2018-06-08,Commander Anthology Volume II,cm2,NaN,312,commander


This looks better - I recognize more of these sets, and the promotional ones are tournament legal.

## Get Card Data

In [7]:
# get 1 page
link = 'https://api.scryfall.com/cards?page=1'
response = requests.get(link)
cards = response.json()['data']
cards_df = pd.DataFrame(cards)

In [8]:
cards_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 55 columns):
all_parts            2 non-null object
artist               175 non-null object
border_color         175 non-null object
card_faces           10 non-null object
cmc                  175 non-null float64
collector_number     175 non-null object
color_identity       175 non-null object
colors               165 non-null object
colorshifted         175 non-null bool
digital              175 non-null bool
edhrec_rank          136 non-null float64
eur                  14 non-null object
flavor_text          78 non-null object
foil                 175 non-null bool
frame                175 non-null object
full_art             175 non-null bool
futureshifted        175 non-null bool
highres_image        175 non-null bool
id                   175 non-null object
illustration_id      112 non-null object
image_uris           165 non-null object
lang                 175 non-null object
layout  

In [9]:
# Filters: not legal in vintage (tokens, joke cards, conspiracies, etc.), only english cards
clean_cards = cards_df[(cards_df['legalities'].apply(lambda x: x['vintage']!='not_legal')) & (cards_df['lang']=='en')]
clean_cards.set_index('id', inplace=True)
clean_cards.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148 entries, ff92804a-0c62-4eb8-bbba-f1ca6f426b6e to 1895cf0c-9c2d-41f9-9819-7348ac9e25f0
Data columns (total 54 columns):
all_parts            0 non-null object
artist               148 non-null object
border_color         148 non-null object
card_faces           0 non-null object
cmc                  148 non-null float64
collector_number     148 non-null object
color_identity       148 non-null object
colors               148 non-null object
colorshifted         148 non-null bool
digital              148 non-null bool
edhrec_rank          136 non-null float64
eur                  9 non-null object
flavor_text          73 non-null object
foil                 148 non-null bool
frame                148 non-null object
full_art             148 non-null bool
futureshifted        148 non-null bool
highres_image        148 non-null bool
illustration_id      95 non-null object
image_uris           148 non-null object
lang                 148 non-n

In [10]:
# Features to keep
MVP_features = [
    'name',
    'set_name',
    'type_line',    
    'mana_cost',
    'rarity',
    'oracle_text',
    'power',
    'toughness',
    'loyalty',
    'cmc',
    'set',
    'color_identity',
    'colors',    
    'reprint',
    'layout',
    'legalities',
]

misc_features = [
    'all_parts',
    'artist',
    'border_color',
    'card_faces',
    'edhrec_rank',
    'flavor_text',
    'foil',
    'nonfoil',
    'full_art',
    'watermark'    
    'timeshifted',
    'colorshifted',
    'futureshifted',
    'illustration_id',
    'multiverse_ids',
    'oracle_id',
    'prints_search_uri',
    'rulings_uri',
    'set_search_uri',
]

In [11]:
clean_cards[MVP_features].head()

,name,set_name,type_line,mana_cost,rarity,oracle_text,power,toughness,loyalty,cmc,set,color_identity,colors,reprint,layout,legalities
id,,,,,,,,,,,,,,,,
ff92804a-0c62-4eb8-bbba-f1ca6f426b6e,"Urborg, Tomb of Yawgmoth",Ultimate Box Topper,Legendary Land,,mythic,Each land is a Swamp in addition to its other ...,NaN,NaN,NaN,0.0,puma,[],[],True,normal,"{'standard': 'not_legal', 'future': 'not_legal..."
5e63fd70-ca5b-45fd-b551-9ebe02410e9c,Stirring Wildwood,Ultimate Box Topper,Land,,mythic,Stirring Wildwood enters the battlefield tappe...,NaN,NaN,NaN,0.0,puma,"[G, W]",[],True,normal,"{'standard': 'not_legal', 'future': 'not_legal..."
54f41726-e0bb-4154-a2db-4b68b50f5032,Raging Ravine,Ultimate Box Topper,Land,,mythic,Raging Ravine enters the battlefield tapped.\n...,NaN,NaN,NaN,0.0,puma,"[G, R]",[],True,normal,"{'standard': 'not_legal', 'future': 'not_legal..."
7c9fb3d9-e018-4aa3-9c14-1a51fae176b4,Lavaclaw Reaches,Ultimate Box Topper,Land,,mythic,Lavaclaw Reaches enters the battlefield tapped...,NaN,NaN,NaN,0.0,puma,"[B, R]",[],True,normal,"{'standard': 'not_legal', 'future': 'not_legal..."
ff790ded-af9f-4e93-84b7-ddadff5ccad4,Karakas,Ultimate Box Topper,Legendary Land,,mythic,{T}: Add {W}.\n{T}: Return target legendary cr...,NaN,NaN,NaN,0.0,puma,[W],[],True,normal,"{'standard': 'not_legal', 'future': 'not_legal..."


Next Step: Write loop into scraper, get all the cards! Get the hell off jupyter ntoebooK!

### Formats
Starting w/ modern:
* Standard
* Modern
* Extended
* Legacy
* Vintage
* Block Constructed (deprecated)
* Extended (deprecated)
* Commander
* Casual

Questions:
    Should I only care about non foils?
    Should I do reprint / set search on my own, or use the API? 

## Post-Scryfall Scrape: How do the cards look?

In [3]:
all_cards_df = pd.read_csv('all_vintage_cards.csv')
all_cards_df.info()

C:\Users\alexg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41291 entries, 0 to 41290
Data columns (total 17 columns):
id                41291 non-null object
name              41291 non-null object
set_name          41291 non-null object
type_line         41291 non-null object
mana_cost         36309 non-null object
rarity            41291 non-null object
oracle_text       40216 non-null object
power             18913 non-null object
toughness         18913 non-null object
loyalty           306 non-null object
cmc               41291 non-null float64
set               41291 non-null object
color_identity    41291 non-null object
colors            41113 non-null object
reprint           41291 non-null bool
layout            41291 non-null object
legalities        41291 non-null object
dtypes: bool(1), float64(1), object(15)
memory usage: 5.1+ MB


## MTGPrice Scraping

In [74]:
# trying slimit parser
from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor

# Turn card data into soup
link = 'https://www.mtgprice.com/sets/Visions/Vampiric_Tutor'
soup = BeautifulSoup(requests.get(link).content, 'html.parser')

# GET RESULTS
text_to_find = 'var results = ['
history=[]
for script in soup.findAll('script', type='text/javascript'):
    if text_to_find in script.text:
        parser = Parser()
        tree = parser.parse(script.text)
        for node in nodevisitor.visit(tree):
            if isinstance(node, ast.Assign) and getattr(node.left, 'value', '') == "\"data\"":
                for prices in node.right.items:
                    history.append([prices.items[0].value,prices.items[1].value])
                break
print(np.array(history).shape)

(1787, 2)


### Let's try a whole set

In [157]:
def card_price_history(setname, cardname):
    # Turn card data into soup
    link = 'https://www.mtgprice.com/sets/' + '_'.join(setname.split()) + '/' + '_'.join(cardname.split())
    soup = BeautifulSoup(requests.get(link).content, 'html.parser')

    # GET RESULTS
    text_to_find = 'var results = ['
    history=[]
    for script in soup.findAll('script', type='text/javascript'):
        if text_to_find in script.text:
            parser = Parser()
            tree = parser.parse(script.text)
            for node in nodevisitor.visit(tree):
                if isinstance(node, ast.Assign) and getattr(node.left, 'value', '') == "\"data\"":
                    for prices in node.right.items:
                        history.append([prices.items[0].value,prices.items[1].value])
                    break
    return np.array(history)

In [130]:
hist_dict['Anvil of Bogardan'][-1]

array(['1541404800000', '14.93'], dtype='<U13')

In [142]:
sets = [all_cards_df['set_name'].unique()[31]]
sets

['Rivals of Ixalan']

In [158]:
set_dict = {}
for setname in sets:
    print(setname)
    cards = all_cards_df[all_cards_df['set_name'] == setname]['name'].values
    card_dict = {}
    for cardname in cards:
        print(cardname)
        history = card_price_history(setname, cardname)
        card_dict[cardname] = history
    set_dict[setname] = card_dict
    
print(set_dict)
        

Rivals of Ixalan
Cinder Barrens


Angrath's Fury


Swab Goblin


Angrath's Ambusher


Angrath, Minotaur Pirate


...

Silent Gravestone


Orazca Relic


The Immortal Sun


Golden Guardian // Gold-Forge Garrison


AttributeError: 'Null' object has no attribute 'items'

In [153]:
all_cards_df[all_cards_df['set_name'] == 'Rivals of Ixalan']['name'].values

array(['Cinder Barrens', "Angrath's Fury", 'Swab Goblin',
       "Angrath's Ambusher", 'Angrath, Minotaur Pirate', "Vraska's Scorn",
       "Vraska's Conquistador", 'Vampire Champion',
       'Vraska, Scheming Gorgon', 'Forest', 'Mountain', 'Swamp', 'Island',
       'Plains', 'Woodland Stream', 'Stone Quarry', 'Highland Lake',
       'Foul Orchard', 'Forsaken Sanctuary', 'Evolving Wilds',
       'Arch of Orazca', "Traveler's Amulet", 'Strider Harness',
       'Silent Gravestone', 'Orazca Relic', 'The Immortal Sun',
       'Golden Guardian // Gold-Forge Garrison', 'Gleaming Barrier',
       "Captain's Hook", "Azor's Gateway // Sanctum of the Sun",
       'Awakened Amalgam', 'Zacama, Primal Calamity',
       'Storm the Vault // Vault of Catlacan', 'Storm Fleet Sprinter',
       'Siegehorn Ceratops', 'Resplendent Griffin', 'Relentless Raptor',
       'Raging Regisaur', 'Protean Raider',
       'Profane Procession // Tomb of the Dusk Rose',
       'Path of Mettle // Metzali, Tower of Trium

In [5]:
from scraper import *

In [23]:
connection = connect_mystic()

In [19]:
results = connection.execute("select * from mythic_price_history")
print('recorded mythic price datapoints:')
count = 0
for r in results:
    count+=1
print(count)

results = connection.execute("select count(distinct cardname) from mythic_price_history")
print('number of mythic cards recorded:')
print(results)
for r in results:
    print(r[0])

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 457, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



recorded mythic price datapoints:
64538
number of mythic cards recorded:
(571,)


In [22]:
results = connection.execute("select * from rare_price_history")
print('recorded rare price datapoints:')
count = 0
for r in results:
    count+=1
print(count)

results = connection.execute("select count(distinct cardname) from rare_price_history")
print('number of rare cards recorded:')
for r in results:
    print(r[0])

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 457, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



recorded rare price datapoints:
112107
number of rare cards recorded:
1745


In [24]:
results = connection.execute("select * from rare_price_history where cardname = 'Emeria, the Sky Ruin'")
for r in results:
    print(r)

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\alexg\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 457, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



('Emeria, the Sky Ruin', 'Commander 2014', '1415772093046', 0.7)
('Emeria, the Sky Ruin', 'Commander 2014', '1416290493046', 0.5)
('Emeria, the Sky Ruin', 'Commander 2014', '1418104893046', 0.6)
('Emeria, the Sky Ruin', 'Commander 2014', '1424066493046', 0.8)
('Emeria, the Sky Ruin', 'Commander 2014', '1425621693046', 1.0)
('Emeria, the Sky Ruin', 'Commander 2014', '1426572093046', 0.9)
('Emeria, the Sky Ruin', 'Commander 2014', '1430892093046', 0.8)
('Emeria, the Sky Ruin', 'Commander 2014', '1430978493046', 1.0)
('Emeria, the Sky Ruin', 'Commander 2014', '1431324093046', 1.3)
('Emeria, the Sky Ruin', 'Commander 2014', '1431842493046', 0.8)
('Emeria, the Sky Ruin', 'Commander 2014', '1432706493046', 0.9)
('Emeria, the Sky Ruin', 'Commander 2014', '1432965693046', 1.5)
('Emeria, the Sky Ruin', 'Commander 2014', '1433052093046', 1.2)
('Emeria, the Sky Ruin', 'Commander 2014', '1433138493046', 1.7)
('Emeria, the Sky Ruin', 'Commander 2014', '1433311293046', 1.3)
('Emeria, the Sky Ruin', 